# STEPS TO FOLLOW
### Iterate through each entity (except EPUK and EPL) and for each currency
### 1) Hedge between above and below for each entity != entity_ccy using the entity_ccy as the back-to-back
### 2) Add those amounts of entity_ccy to the corresponding above/below of the entity ccy
### 3) Move the excess from the entity to EPUK, using the entity_ccy as back-to-back
### 4) Add the amount of these movements of entity_ccy to the entity's exposure in the entity_ccy in the above/below
### Now, all the currencies for each entity are hedged, except for the entity_ccy. The entity_ccy exposure isn't moved to EPUK
### 6) Now, for EPUK, hedge between above and below for each entity != entity_ccy using the entity_ccy as the back to back
### 7) Hedge the excess of each currency != entity_ccy in the market with EPUK
### 8) For each entity_ccy in each entity_ccy, hedge the excess to the entity_threshold with EPUK (without moving the exposure to EPUK)
### 9) These last hedges need to be moved to EPL so EPUK isn't exposed to that currency, so last step is to do a internal EPUK<->EPL

In [1]:
import pandas as pd
import numpy as np
import queries as qy

In [2]:
import os
from configparser import ConfigParser

def get_config():
    config = ConfigParser()
    config.read(os.path.join("C:/Users/andres.mireles_ebury/Desktop/Projects/FX Exposure/fx_exposure/config.ini"))

    return config

from bq_link import get_bq_link
bq_client, _ = get_bq_link(get_config())

Load all the exposures

In [3]:
balance_date = "2024-04-30"
exposures_entities = bq_client.query(qy.net_exposure.format(date=balance_date)).drop_duplicates()

Set the group threshold and the individual entity threshold (in the future, these can be weighted)

The total exposures by currency

In [4]:
total_exposures = exposures_entities.groupby("currency")[["above_exposure_gbp","above_exposure_local_ccy","below_exposure_gbp","below_exposure_local_ccy","net_exposure_gbp","net_exposure_local_ccy"]].sum()

Use only the entitiies with high exposures

In [5]:
group_threshold = 5e6

In [6]:
large_exposure_currencies = total_exposures[
    (np.abs(total_exposures.above_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.below_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.net_exposure_gbp)>=group_threshold)
].index

In [7]:
large_exposure_currencies

Index(['AED', 'AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'HKD', 'PLN', 'USD'], dtype='object', name='currency')

In [8]:
exposures_entities = exposures_entities[exposures_entities.currency.isin(large_exposure_currencies)].reset_index(drop=True)

In [9]:
exposures_entities

,currency,entity,entity_ccy,above_exposure_gbp,above_exposure_local_ccy,above_exposure_entity_ccy,below_exposure_gbp,below_exposure_local_ccy,below_exposure_entity_ccy,net_exposure_gbp,net_exposure_local_ccy,net_exposure_entity_ccy
0,CAD,EPUK,GBP,586.57,1009.03,5.865729e+02,-3.635368e+06,-6.253606e+06,-3.635368e+06,-3.634782e+06,-6.252597e+06,-3.634782e+06
1,USD,EPPTE,SGD,0.00,0.00,0.000000e+00,5.093166e+04,6.372569e+04,8.691228e+04,5.093166e+04,6.372569e+04,8.691228e+04
2,HKD,EPCA,CAD,0.01,0.00,0.000000e+00,-4.302142e+04,-4.210000e+05,-7.400598e+04,-4.302141e+04,-4.210000e+05,-7.400598e+04
3,AUD,EPPTE,SGD,0.00,0.00,0.000000e+00,6.661874e+03,1.284517e+04,1.136815e+04,6.661874e+03,1.284517e+04,1.136815e+04
4,EUR,EPPTE,SGD,0.00,0.00,0.000000e+00,1.636307e+04,1.916274e+04,2.792274e+04,1.636307e+04,1.916274e+04,2.792274e+04
...,...,...,...,...,...,...,...,...,...,...,...,...
105,EUR,ETL,GBP,0.00,0.00,0.000000e+00,-2.701334e+07,-3.163524e+07,-2.701334e+07,-2.701334e+07,-3.163524e+07,-2.701334e+07
106,USD,EPL,GBP,0.00,0.00,0.000000e+00,-9.020751e+07,-1.128676e+08,-9.020751e+07,-9.020751e+07,-1.128676e+08,-9.020751e+07
107,AUD,EPF,GBP,-0.01,0.00,-5.820766e-11,8.461003e+05,1.631418e+06,8.461003e+05,8.461003e+05,1.631418e+06,8.461003e+05
108,GBP,EPHK,HKD,11136.29,11136.29,1.089778e+05,7.287082e+06,7.287082e+06,7.131010e+07,7.298219e+06,7.298219e+06,7.141907e+07


Only the entities with large exposures to each currency

In [10]:
entity_threshold = group_threshold / (2 * exposures_entities.entity.unique().shape[0])

In [11]:
exposures_entities = exposures_entities.set_index(["currency","entity","entity_ccy"])

In [12]:
large_exposure_entities_currencies = exposures_entities[
    (np.abs(exposures_entities.above_exposure_gbp)>=entity_threshold)|
    (np.abs(exposures_entities.below_exposure_gbp)>=entity_threshold)|
    (np.abs(exposures_entities.net_exposure_gbp)>=entity_threshold)
].index

In [13]:
exposures_entities = exposures_entities.loc[large_exposure_entities_currencies]

In [94]:
def net_exposure(currency,entity,entity_ccy, above,above_local_ccy,above_entity_ccy,below,below_local_ccy,below_entity_ccy,threshold):

    if above == below == 0 or entity in ("EPUK","EPL") or currency == entity_ccy:
        return above, above_local_ccy, above_entity_ccy, below, below_local_ccy, below_entity_ccy, above+below, above_local_ccy+below_local_ccy, above_entity_ccy+below_entity_ccy, 0, 0, 0, np.nan, np.nan, 0, 0, 0, np.nan
    
    
    # if currency == entity_ccy:
    #     return 

    # Rate to convert gbp to local ccy
    if below != 0:
        rate_to_local_ccy = below_local_ccy / below
        rate_to_entity_ccy = below_entity_ccy / below
    else:
        rate_to_local_ccy = above_local_ccy / above
        rate_to_entity_ccy = above_entity_ccy / above


    # Signs of the below to move the exposure (if below is 0, then just the opposite of the sign of the above)
    if below != 0:
        sign_below = below / abs(below)
    else:
        sign_below = -1 * above / abs(above)

    # The internal change that we need
    internal_change = np.maximum(abs(below) - threshold,0)

    internal_change_local_ccy = internal_change * rate_to_local_ccy
    internal_change_entity_ccy = internal_change * rate_to_entity_ccy

    # The resulting exposures
    new_above = above + sign_below*internal_change
    new_above_local_ccy = above_local_ccy + sign_below*internal_change_local_ccy
    new_above_entity_ccy = above_entity_ccy + sign_below*internal_change_entity_ccy
    new_below = below - sign_below*internal_change
    new_below_local_ccy = below_local_ccy - sign_below*internal_change_local_ccy
    new_below_entity_ccy = below_entity_ccy - sign_below*internal_change_entity_ccy

    # The direction of the internal entity hedges
    below_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"LHS","RHS"),
        np.nan
    )
    above_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"RHS","LHS"),
        np.nan
    )

    # What we are going to move from the above of the entity to the above in epuk
    above_to_epuk = np.maximum(abs(new_above) - threshold,0)

    above_to_epuk_local_ccy = above_to_epuk * rate_to_local_ccy
    above_to_epuk_entity_ccy = above_to_epuk * rate_to_entity_ccy
    
    # The direction of the above to epuk hedges (if above = 0, we don't move anything)
    above_to_epuk_lhs_rhs = np.where(
        above_to_epuk != 0,
        np.where(new_above/abs(new_above)>0,"LHS","RHS"),
        np.nan
    )

    # Update the new above of that entity reducing what we move to epuk
    new_above = new_above - np.where(
        new_above!=0,
        new_above/abs(new_above)*above_to_epuk,
        0
    )
    new_above_local_ccy = new_above_local_ccy - np.where(
        new_above!=0,
        new_above_local_ccy/abs(new_above_local_ccy)*above_to_epuk_local_ccy,
        0
    )
    new_above_entity_ccy = new_above_entity_ccy - np.where(
        new_above!=0,
        new_above_entity_ccy/abs(new_above_entity_ccy)*above_to_epuk_entity_ccy,
        0
    )

    # New net
    new_net = new_below + new_above
    new_net_local_ccy = new_below_local_ccy + new_above_local_ccy
    new_net_entity_ccy = new_below_entity_ccy + new_above_entity_ccy
    


    return new_above, new_above_local_ccy, new_above_entity_ccy, new_below, new_below_local_ccy, new_below_entity_ccy, new_net, new_net_local_ccy, new_net_entity_ccy, internal_change, internal_change_local_ccy, internal_change_entity_ccy, above_lhs_rhs, below_lhs_rhs, above_to_epuk, above_to_epuk_local_ccy, above_to_epuk_entity_ccy, above_to_epuk_lhs_rhs


In [95]:
exposures_entities.head()

above_exposure_gbp  above_exposure_local_ccy  \
currency entity     entity_ccy                                                 
CAD      EPUK       GBP                     586.57                   1009.03   
GBP      EPL        GBP                       0.00                      0.00   
         EPPTE      SGD                       0.00                      0.00   
         EPBE       EUR                  827598.89                 827598.89   
EUR      EPM-Cyprus EUR                       0.00                      0.00   

                                above_exposure_entity_ccy  below_exposure_gbp  \
currency entity     entity_ccy                                                  
CAD      EPUK       GBP                        586.572937       -3.635368e+06   
GBP      EPL        GBP                          0.000000        6.285352e+07   
         EPPTE      SGD                          0.000000       -8.136518e+05   
         EPBE       EUR                     969198.550326        7.409683e+07   
EUR      EPM-Cyprus EUR                          0.000000       -2.260228e+05   

                                below_exposure_local_ccy  \
currency entity     entity_ccy                             
CAD      EPUK       GBP                      -6253605.56   
GBP      EPL        GBP                      62853520.36   
         EPPTE      SGD                       -813651.79   
         EPBE       EUR                      74096834.91   
EUR      EPM-Cyprus EUR                       -264694.64   

                                below_exposure_entity_ccy  net_exposure_gbp  \
currency entity     entity_ccy                                                
CAD      EPUK       GBP                     -3.635368e+06     -3.634782e+06   
GBP      EPL        GBP                      6.285352e+07      6.285352e+07   
         EPPTE      SGD                     -1.388455e+06     -8.136518e+05   
         EPBE       EUR                      8.677458e+07      7.492443e+07   
EUR      EPM-Cyprus EUR                     -2.646946e+05     -2.260228e+05   

                                net_exposure_local_ccy  \
currency entity     entity_ccy                           
CAD      EPUK       GBP                    -6252596.53   
GBP      EPL        GBP                    62853520.36   
         EPPTE      SGD                     -813651.79   
         EPBE       EUR                    74924433.80   
EUR      EPM-Cyprus EUR                     -264694.64   

                                net_exposure_entity_ccy  
currency entity     entity_ccy                           
CAD      EPUK       GBP                   -3.634782e+06  
GBP      EPL        GBP                    6.285352e+07  
         EPPTE      SGD                   -1.388455e+06  
         EPBE       EUR                    8.774378e+07  
EUR      EPM-Cyprus EUR                   -2.646946e+05

## Step 1)

In [205]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

exposures_entities_net = pd.DataFrame()

for entity in exposures_entities.index.get_level_values(1).unique():
    
    exposure_ent = exposures_entities.xs(entity, level=1)

    exposure_ent[
        [
            'new_above_exposure_gbp', 
            'new_above_exposure_local_ccy', 
            'new_above_exposure_entity_ccy', 
            'new_below_exposure_gbp', 
            'new_below_exposure_local_ccy', 
            'new_below_exposure_entity_ccy', 
            'new_net_gbp', 
            'new_net_local_ccy', 
            'new_net_entity_ccy', 
            'internal_change_gbp', 
            'internal_change_local_ccy', 
            'internal_change_entity_ccy', 
            'above_lhs_rhs', 
            'below_lhs_rhs', 
            'above_to_epuk_gbp', 
            'above_to_epuk_local_ccy', 
            'above_to_epuk_entity_ccy', 
            'above_to_epuk_lhs_rhs'
        ]
    ] = exposure_ent.apply(
        lambda row: pd.Series(
            net_exposure(
                row.name[0],
                entity, 
                row.name[1], 
                row['above_exposure_gbp'], 
                row['above_exposure_local_ccy'], 
                row['above_exposure_entity_ccy'], 
                row['below_exposure_gbp'], 
                row['below_exposure_local_ccy'], 
                row['below_exposure_entity_ccy'], 
                entity_threshold
            )
        ),
        axis=1
    )

    exposure_ent["entity"] = entity
    exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])
    
    exposures_entities_net = pd.concat([exposures_entities_net,exposure_ent])
    

In [210]:
exposure_ent[[c for c in exposure_ent.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
HKD,EPHK,HKD,18472.90,-5.329014e+05,-5.144285e+05,-3.060867e+06,2.546438e+06,-514428.473497,0.000000e+00,NaN,NaN,0.000000e+00,NaN
EUR,EPHK,HKD,628697.82,-7.764403e+05,-1.477425e+05,-6.836305e+02,-1.470588e+05,-147742.453992,6.293815e+05,LHS,RHS,0.000000e+00,nan
USD,EPHK,HKD,-21877.70,-3.578362e+06,-3.600239e+06,-1.470588e+05,-1.470588e+05,-294117.647059,3.431303e+06,LHS,RHS,3.306122e+06,RHS
GBP,EPHK,HKD,11136.29,7.287082e+06,7.298219e+06,1.470588e+05,1.470588e+05,294117.647059,7.140024e+06,RHS,LHS,7.004101e+06,LHS


## Step 2)

In [207]:
for entity in exposures_entities_net.index.get_level_values(1).unique():

    
    if entity not in ("EPUK","EPL"):
        exposure_ent = exposures_entities_net.xs(entity, level=1)

        # Locate the entity_ccy exposure to add the exposure from the internal changes (to the above and below)
        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_above_exposure_gbp",
            "new_above_exposure_local_ccy",
            "new_above_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["above_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
        ).sum(axis=0)

        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_below_exposure_gbp",
            "new_below_exposure_local_ccy",
            "new_below_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["below_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
        ).sum(axis=0)

        exposure_ent["entity"] = entity
        exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])

In [209]:
exposure_ent[[c for c in exposure_ent.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
HKD,EPHK,HKD,18472.90,-5.329014e+05,-5.144285e+05,-3.060867e+06,2.546438e+06,-514428.473497,0.000000e+00,NaN,NaN,0.000000e+00,NaN
EUR,EPHK,HKD,628697.82,-7.764403e+05,-1.477425e+05,-6.836305e+02,-1.470588e+05,-147742.453992,6.293815e+05,LHS,RHS,0.000000e+00,nan
USD,EPHK,HKD,-21877.70,-3.578362e+06,-3.600239e+06,-1.470588e+05,-1.470588e+05,-294117.647059,3.431303e+06,LHS,RHS,3.306122e+06,RHS
GBP,EPHK,HKD,11136.29,7.287082e+06,7.298219e+06,1.470588e+05,1.470588e+05,294117.647059,7.140024e+06,RHS,LHS,7.004101e+06,LHS


## Step 3)

In [ ]:
exposures_entities.loc[("EPUK","GBP"),"new_above_gbp"] = exposures_entities.loc[("EPUK","GBP"),"new_above_gbp"] + np.sum(
    np.where(
        exposures_entities[exposures_entities.index!="EPUK"]["above_to_EPUK_lhs_rhs"] == "RHS",
        -1 * exposures_entities[exposures_entities.index!="EPUK"]["above_to_EPUK_gbp"],
        exposures_entities[exposures_entities.index!="EPUK"]["above_to_EPUK_gbp"],
    )
)

exposures_entities.loc[("EPUK","GBP"),"new_above_local_ccy"] = exposures_entities.loc[("EPUK","GBP"),"new_above_local_ccy"] + np.sum(
    np.where(
        exposures_entities[exposures_entities.index!="EPUK"]["above_to_EPUK_lhs_rhs"] == "RHS",
        -1 * exposures_entities[exposures_entities.index!="EPUK"]["above_to_EPUK_local_ccy"],
        exposures_entities[exposures_entities.index!="EPUK"]["above_to_EPUK_local_ccy"],
    )
)

exposures_entities.loc[("EPUK","GBP"),"new_net_gbp"] = exposures_entities.loc[("EPUK","GBP"),"new_above_gbp"] + exposures_entities.loc[("EPUK","GBP"),"new_below_gbp"]
exposures_entities.loc[("EPUK","GBP"),"new_net_local_ccy"] = exposures_entities.loc[("EPUK","GBP"),"new_above_local_ccy"] + exposures_entities.loc[("EPUK","GBP"),"new_below_local_ccy"]

In [ ]:
exposures_entities["net_exposure_gbp"].sum()

In [ ]:
exposures_entities["new_net_gbp"].sum()

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1,2,figsize=(15,5))

axs[0].bar(x=exposures_entities["above_exposure_gbp"].index.get_level_values(0),height=exposures_entities["above_exposure_gbp"])
axs[0].bar(x=exposures_entities["below_exposure_gbp"].index.get_level_values(0),height=exposures_entities["below_exposure_gbp"])
axs[0].legend(["Above Exposure (GBP)","Below Exposure (GBP)"])
axs[0].set_title("Pre Exposure")
axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

axs[1].bar(x=exposures_entities["new_above_gbp"].index.get_level_values(0),height=exposures_entities["new_above_gbp"])
axs[1].bar(x=exposures_entities["new_below_gbp"].index.get_level_values(0),height=exposures_entities["new_below_gbp"])
axs[1].legend(["New Above Exposure (GBP)","New Below Exposure (GBP)"])
axs[1].set_title("New Exposure")
axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1,2,figsize=(15,5))

axs[0].bar(x=exposures_entities["net_exposure_gbp"].index.get_level_values(0),height=exposures_entities["net_exposure_gbp"],color="green")
axs[0].legend(["Net Exposure (GBP)"])
axs[0].set_title("Pre Exposure")
axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

axs[1].bar(x=exposures_entities["new_net_gbp"].index.get_level_values(0),height=exposures_entities["new_net_gbp"],color="green")
axs[1].legend(["New Net Exposure (GBP)"])
axs[1].set_title("New Exposure")
axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
plt.show()

### New, we net EPUK and hedge external if needed:

In [ ]:
hedges_EPUK = exposures_entities.loc[("EPUK","GBP")].rename("EPUK_hedges").to_frame().T.apply(
    lambda row: pd.Series(
        net_exposure(
            currency,
            row.name[0], 
            row.name[1], 
            row['above_exposure_gbp'], 
            row['above_exposure_local_ccy'], 
            row['above_exposure_entity_ccy'], 
            row['below_exposure_gbp'], 
            row['below_exposure_local_ccy'], 
            row['below_exposure_entity_ccy'], 
            entity_threshold
        )
    ),
    axis=1
)

hedges_EPUK.columns = [
    'new_above_gbp', 
    'new_above_local_ccy', 
    'new_above_entity_ccy', 
    'new_below_gbp', 
    'new_below_local_ccy', 
    'new_below_entity_ccy', 
    'new_net_gbp', 
    'new_net_local_ccy', 
    'new_net_entity_ccy', 
    'internal_change_gbp', 
    'internal_change_local_ccy', 
    'internal_change_entity_ccy', 
    'above_lhs_rhs', 
    'below_lhs_rhs', 
    'above_external_gbp', 
    'above_external_local_ccy', 
    'above_external_entity_ccy', 
    'above_external_lhs_rhs'
]

multi_index = pd.MultiIndex.from_product([["EPUK"], ["GBP"]])

hedges_EPUK.index = multi_index

hedges_EPUK

In [ ]:
pd.concat(
    [
        exposures_entities.loc[("EPUK","GBP")][['above_exposure_gbp','above_exposure_local_ccy','above_exposure_entity_ccy','below_exposure_gbp','below_exposure_local_ccy','below_exposure_entity_ccy','net_exposure_gbp','net_exposure_local_ccy','net_exposure_entity_ccy']].to_frame().T,
        hedges_EPUK[["new_above_gbp","new_above_local_ccy","new_above_entity_ccy","new_below_gbp","new_below_local_ccy","new_below_entity_ccy","new_net_gbp","new_net_local_ccy","new_net_entity_ccy"]]
    ],
    axis=1
)